In [1]:
pip install dash

Note: you may need to restart the kernel to use updated packages.


In [2]:
import json
import threading
import time
from collections import deque
from kafka import KafkaConsumer
import dash
from dash import dcc, html
from dash.dependencies import Output, Input

In [3]:
# ----------------------------
# Kafka Consumer Setup
# ----------------------------
consumer = KafkaConsumer(
    'sensor-data',
    bootstrap_servers=['localhost:9092'],
    value_deserializer=lambda m: json.loads(m.decode('utf-8')),
    auto_offset_reset='latest',
    enable_auto_commit=True,
    group_id='dashboard-group'
)

In [4]:
# ----------------------------
# Store recent data in memory
# ----------------------------
max_len = 50  # keep last 50 points
temps, hums, times = deque(maxlen=max_len), deque(maxlen=max_len), deque(maxlen=max_len)

def consume_kafka():
    for msg in consumer:
        data = msg.value
        temps.append(data["temperature"])
        hums.append(data["humidity"])
        times.append(data["timestamp"])

# Run Kafka consumer in background thread
threading.Thread(target=consume_kafka, daemon=True).start()

In [5]:
# ----------------------------
# Dash App Setup
# ----------------------------
app = dash.Dash(__name__)
app.layout = html.Div(
    [
        html.H1("🌡️ Real-Time Sensor Dashboard", style={"textAlign": "center"}),
        dcc.Graph(id="live-graph", animate=True),
        dcc.Interval(id="update-interval", interval=2000, n_intervals=0),  # update every 2s
    ]
)

In [6]:
@app.callback(Output("live-graph", "figure"), [Input("update-interval", "n_intervals")])
def update_graph(n):
    import plotly.graph_objs as go

    return {
        "data": [
            go.Scatter(x=list(times), y=list(temps), name="Temperature (°C)", mode="lines+markers"),
            go.Scatter(x=list(times), y=list(hums), name="Humidity (%)", mode="lines+markers"),
        ],
        "layout": go.Layout(
            xaxis=dict(title="Timestamp"),
            yaxis=dict(title="Value"),
            title="Live Sensor Data Stream",
            showlegend=True,
            uirevision='constant'
        ),
    }

In [7]:
if __name__ == "__main__":
    print("🚀 Starting Dashboard at http://127.0.0.1:8050")
    app.run(debug=True)

🚀 Starting Dashboard at http://127.0.0.1:8050
